# Mount drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# !ls /content/gdrive/My\ Drive

In [ ]:
# !unzip /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319/hw1_data.zip -d /content/gdrive/MyDrive/Q56094077/snrs/hw1_0319

# Import Library

In [37]:
import os

import torch
import torch.nn as nn

import pandas as pd
import numpy as np
import json

from tqdm import tqdm
from datetime import datetime

In [38]:
import torch_geometric
from torch_geometric.data import Data, DataLoader
import torch_geometric.utils as utils

In [39]:
import networkit as nk

In [40]:
import networkx as nx

# Setting

In [41]:
class Setting:
    _root = os.getcwd()

    _data = os.path.join(_root, "hw1_data")

    data_synthetic = os.path.join(_data, "Synthetic", "5000")
    data_youtube = os.path.join(_data, "youtube")
    
    
     # Create dir for train/test
    date_time = datetime.strftime(datetime.now(), "%Y-%m-%d %H-%M")
    root = os.path.join(_root, "result", date_time)
    if os.path.exists(root):
        pass
    else:
        os.makedirs(root)


    ### Save plt info
    train_info_p = os.path.join(root, "train.json")
    val_info_p = os.path.join(root, "valid.json")
    test_info_p = os.path.join(root, "test.json")

    ### Save plt img
    result_plt_p = os.path.join(root, "train_plt.png")
    test_plt_p = os.path.join(root, "test_plt.png")
    sum_box_p = os.path.join(root, "sum_box.png")
        
        
    ### Data split
    data_split = os.path.join(_root, "split.json")
    

    # Setting of training
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

    mini_epochs = 500
    epochs = 10000

    criterion = nn.BCELoss()
    weight_node2vec = os.path.join(root, "node2vec.pth")
    weight_drbc = os.path.join(root, "drbc.pth")
    
    params_drbc = None
    params_node2vec = None

    

In [42]:
setting = Setting()

# Dataset

## Data

- data.x	节点特征，维度是[num_nodes, num_node_features]。
- data.edge_index	维度是[2, num_edges]，描述图中节点的关联关系，每一列对应的两个元素，分别是边的起点和重点。数据类型是torch.long。需要注意的是，data.edge_index是定义边的节点的张量（tensor），而不是节点的列表（list）。
- data.edge_attr	边的特征矩阵，维度是[num_edges, num_edge_features]
- data.y	训练目标（维度可以是任意的）。对于节点相关的任务，维度为[num_nodes, *]；对于图相关的任务，维度为[1,*]。
- data.position	节点位置矩阵（Node position matrix），维度为[num_nodes, num_dimensions]。

- [Learning to Identify High Betweenness Centrality Nodes from
Scratch: A Novel Graph Neural Network Approach](https://arxiv.org/pdf/1905.10418.pdf)
- node initial feature = [$(d_v), 1, 1]

### Load all data

In [43]:
def load_data(path):

    edge_index = []
    between = []
    for f in os.listdir(path):
        if "score" in f:
            # ground truth of betweenness centrality
            p = os.path.join(path, f)
            between.append(p)
            pass
        else:
            p = os.path.join(path, f)
            edge_index.append(p)

    between.sort()
    edge_index.sort()
    
    return edge_index, between
    
    
synthetic, between = load_data(setting.data_synthetic)
yt, yt_score = load_data(setting.data_youtube)

### Split Data

In [44]:
from sklearn.model_selection import train_test_split


def split_data(path, x, y, replace=False):
    if os.path.exists(path) and replace:
        pass
    else:
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
        X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15)

        split = {
            "X_train": X_train,
            "X_valid": X_valid,
            "X_test": X_test,
            "y_train": y_train,
            "y_valid": y_valid,
            "y_test": y_test
        }

        with open(path, 'w') as f:
            json.dump(split, f)
            
split_data(path=setting.data_split, x=synthetic, y=between)

### Cvt 2 dataloader

In [45]:
def to_dataloader(x, y, batch):
    
    data_list = []
    for index, f in enumerate(x):
        edge_index = torch_geometric.io.read_txt_array(f, dtype=torch.long)
        edge_index = edge_index.t().contiguous()
        edge_index = utils.to_undirected(edge_index)

        row, col = edge_index  
        deg = utils.degree(col) # must use col to get degree, why?
        deg = deg.numpy()  

        vertice = []
        for d in deg:
            vertice.append([d, 1, 1])
        vertice = np.array(vertice, dtype=np.float)
        vertice = torch.from_numpy(vertice)

        ### between centrality
        bcs = []
        bc = torch_geometric.io.read_txt_array(y[index], dtype=torch.double)
        bc = bc.t().contiguous()
        row, col = bc
        bc = col
        bc = bc.numpy()
        
        bcs = torch.from_numpy(
                    np.array([[b] for b in bc], dtype=np.float))

        data = Data(x=vertice, edge_index=edge_index, y=bcs)
        data_list.append(data)

    loader = DataLoader(data_list, batch_size=batch)
    return data_list

### Cvt 2 Data

In [46]:
def to_data(x, y):
    edge_index = torch_geometric.io.read_txt_array(x, dtype=torch.long)
    edge_index = edge_index.t().contiguous()
    edge_index = utils.to_undirected(edge_index)

    row, col = edge_index  
    deg = utils.degree(col) # must use col to get degree, why?
    deg = deg.numpy()  

    vertice = []
    for d in deg:
        vertice.append([d, 1, 1])
    vertice = np.array(vertice, dtype=np.float)
    vertice = torch.from_numpy(vertice)

    ### between centrality
    bcs = []
    bc = torch_geometric.io.read_txt_array(y, dtype=torch.double)
    bc = bc.t().contiguous()
    row, col = bc
    bc = col
    bc = bc.numpy()
    bcs = torch.from_numpy(
            np.array([[b] for b in bc], dtype=np.float))

    data = Data(x=vertice, edge_index=edge_index, y=bcs)
    
    return data

#### Read data

In [47]:
f = open(setting.data_split, 'r')
data_split = json.load(f)

X_train = data_split["X_train"]
X_valid = data_split["X_valid"]
X_test = data_split["X_test"]
y_train = data_split["y_train"]
y_valid = data_split["y_valid"]
y_test = data_split["y_test"]

f.close()

#### Cvt

In [48]:
train_loader = to_dataloader(X_train, y_train, batch=1)
valid_loader = to_dataloader(X_valid, y_valid, batch=1)
test_loader = to_dataloader(X_test, y_test, batch=1)

#### Use all data

In [49]:
all_loader = to_dataloader(synthetic, between, batch=1)

#### Use YT data

In [50]:
# yt_loader = to_dataloader(yt, yt_score, batch=1)

# Model

## Hyper params

In [51]:
setting.params_drbc = dict(
    # according to source paper
    encoder_params = dict(
        c = 3,
        p = 128,
        num_layers = 5,
        device = setting.device
    ),
    decoder_params = dict(
        p = 128,
        q = 64
    )
)

# setting.params_node2vec = dict(
#     edge_index, 
#     embedding_dim, 
#     walk_length = 10, 
#     context_size,
#     p = 1,
#     q = 2
# )

## DrBC

In [52]:
from torch_geometric.nn import MessagePassing
import torch.nn.functional as F
from torch_geometric.typing import Adj, OptTensor

In [53]:
class Encoder(MessagePassing):
    def __init__(self, c, p, num_layers, device, aggr="add"):
        super(Encoder, self).__init__(aggr=aggr)
        
        self.num_layers = num_layers
        self.w_0 = nn.Linear(in_features=c, out_features=p).double()
        
        self.relu = nn.ReLU(inplace=True)
        self.rnn = nn.GRUCell(p, p).double()

        self.device = device

    def forward(self, data):
        
        x, edge_index = data.x, data.edge_index
        
        
        # compute dgree
        row, col = edge_index
        deg = utils.degree(col)
        deg = torch.add(deg, 1)
        deg_inv_sqrt = torch.pow(deg, -0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]
        
        
        # h_0 = x

        # h_1
        x = self.w_0(x)
        x = self.relu(x)
        x = F.normalize(x, p=2, dim=1)
        
        h_s = [x]
        
        for i in range(self.num_layers-1):
            # internally calls the message(), aggregate() and update() functions
            x = self.propagate(edge_index, x=x, norm=norm)
            h_s.append(x)
        
        h_s = torch.stack(h_s, dim=-1)
        # Use torch.max to replace max_pooling
        z, _ = torch.max(h_s, dim=-1)
        # z = global_max_pool(h_s, torch.tensor([0], dtype=torch.long).to(self.device))
        
        return z

    def message(self, x_j, norm: OptTensor):
        """      
        In addition, tensors passed to propagate() can be mapped to the respective nodes i and j 
        by appending _i or _j to the variable name, .e.g. x_i and x_j. 
        Note that we generally refer to i as the central nodes that aggregates information, 
        and refer to j as the neighboring nodes, since this is the most common notation.
        """
  
        return x_j if norm is None else norm.view(-1, 1) * x_j
    
    
    def update(self, aggr_out, x):
        """        
        Takes in the output of aggregation as first argument 
        and any argument which was initially passed to propagate().
        """

        x = self.rnn(x, aggr_out)
        x = F.normalize(x, p=2, dim=1) 
        
        return x
    

In [54]:
class Decoder(nn.Module):
    def __init__(self, p, q):
        
        super().__init__()
        
        self.w_4 = nn.Linear(in_features=p, out_features=q).double()
        self.w_5 = nn.Linear(in_features=q, out_features=1).double()
        
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, z):
        z = self.w_4(z)
        z = self.relu(z)
        z = self.w_5(z)
        
        return z

In [55]:
class DrBC(nn.Module):
    def __init__(self, encoder_params, decoder_params):
        super().__init__()
        
        self.encoder = Encoder(**encoder_params)
        self.decoder = Decoder(**decoder_params)
        
        
    def forward(self, data):
        
        z = self.encoder(data)
        
        return self.decoder(z)

## Node2Vec

In [56]:
from torch_geometric.nn import Node2Vec

In [65]:
class BC_node2vec(nn.Module):
    
    def __init__(self, node2vec_params):
        super().__init__()
        
        self.node2vec = Node2Vec(**node2vec_params)
        self.mlp = nn.Linear(in_features=128, out_features=1)
        
    def forward(self, data):
        
        x = self.node2vec(data)
        x = self.mlp(x)
        
        return x

# Load model

In [66]:
def load_checkpoint(filepath, device, **params):

    if "node2vec" in filepath:
        model = BC_node2vec(**params["node2vec"])
    
    else:
        model = DrBC(**params["drbc"])
    
    model = model.to(setting.device)
    
    if os.path.exists(filepath):
        print("pretrained finded")
        checkpoint = torch.load(filepath)
        model.load_state_dict(checkpoint['model_stat'])
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        optimizer.load_state_dict(checkpoint['optimizer_stat'])

    else:
        print("use a new optimizer")
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    return model, optimizer

# Evaluation

### Top k %

In [67]:
def top_n(model, data, k):

    with torch.no_grad():

        data = data.to(setting.device)

        bc_gt = data.y
        bc_gt = torch.reshape(bc_gt, (-1, ))
        
        nodes = bc_gt.size()[0]
        k = int(nodes * k / 100)

        bc_pr = model(data)

        bc_gt = bc_gt.squeeze().to(setting.device)
        bc_pr = bc_pr.squeeze()

        gt_value, gt_indice = torch.topk(bc_gt, k)
        pr_value, pr_indice = torch.topk(bc_pr, k)

        gt_indice = set(gt_indice.cpu().numpy())
        pr_indice = set(pr_indice.cpu().numpy())

        intersect = len(gt_indice & pr_indice)
        top = intersect/k
            
       

        return top

### Tau Kendal

In [68]:
from scipy import stats

def tau_distance(model, data):
    
    
    with torch.no_grad():
 
        data = data.to(setting.device)

        bc_gt = data.y
        bc_gt = torch.reshape(bc_gt, (-1, ))

        bc_pr = model(data)

        bc_gt = bc_gt.squeeze().cpu().numpy()
        bc_pr = bc_pr.squeeze().cpu().numpy()


        tau, p_value = stats.kendalltau(bc_gt, bc_pr)
       

    return tau

### Evaluation Mode

In [69]:
def eval_model(model, data):
    
    with torch.no_grad():
        data = data.to(setting.device)
        bc_pr = model(data)

        bc_gt = data.y
        bc_gt = torch.reshape(bc_gt, (-1, ))
        
        nodes = bc_gt.size()[0]

        # random sample 5|V| nodes
        src = (torch.rand(nodes * 5) * (nodes-1)).long()
        det = (torch.rand(nodes * 5) * (nodes-1)).long()

        y_gt = (bc_gt[det] - bc_gt[src]).squeeze().to(setting.device)
        y_pr = (bc_pr[det] - bc_pr[src]).squeeze()

        y_gt = nn.Sigmoid()(y_gt)
        y_pr = nn.Sigmoid()(y_pr)

        loss = nn.BCELoss(reduction="sum")(y_pr, y_gt)
        
        return loss.item()

In [73]:
def from_networkx(G, between_list):
    r"""Converts a :obj:`networkx.Graph` or :obj:`networkx.DiGraph` to a
    :class:`torch_geometric.data.Data` instance.

    Args:
        G (networkx.Graph or networkx.DiGraph): A networkx graph.
    """

    G = nx.convert_node_labels_to_integers(G)
    G = G.to_directed() if not nx.is_directed(G) else G
    edge_index = torch.LongTensor(list(G.edges)).t().contiguous()
   
    data = {}

    for i, (_, feat_dict) in enumerate(G.nodes(data=True)):
        for key, value in feat_dict.items():
           
            data[str(key)] = [value] if i == 0 else data[str(key)] + [value]

    for i, (_, _, feat_dict) in enumerate(G.edges(data=True)):
        for key, value in feat_dict.items():
            data[str(key)] = [value] if i == 0 else data[str(key)] + [value]

    for key, item in data.items():
        try:
            data[key] = torch.tensor(item)
        except ValueError:
            pass

    data['edge_index'] = edge_index.view(2, -1)
    data['x'] = torch.from_numpy(
        np.array( [ [G.degree[i], 1, 1] for i in G.nodes()], dtype=np.float ) )
    data['y'] = torch.from_numpy(
        np.array( [ [b] for b in between_list ] , dtype=np.float) )
    data = torch_geometric.data.Data.from_dict(data)
    data.num_nodes = G.number_of_nodes()

    return data

# Train

In [74]:
model, optimizer = load_checkpoint(
                    setting.weight_drbc,
                    setting.device,
                    drbc = setting.params_drbc,
                    node2vec = setting.params_node2vec)

use a new optimizer


In [ ]:
nodes_cnt = 3000


model.train()

train_info = {
    "bce": [],
    "top1": [],
    "top5": [],
    "top10": [],
    "kendal": []
}

valid_info = {
    "bce": [],
    "top1": [],
    "top5": [],
    "top10": [],
    "kendal": []
}

top_10 = 0.0
top_5 = 0.0
top_1 = 0.0

for epoch in range(setting.epochs):
    
    # Draw network G from distribution D (like the power-law model)
    G = nx.generators.random_graphs.powerlaw_cluster_graph(n=nodes_cnt, m=4, p=0.05)
    # Calculate each node’s exact BC value bv, ∀v ∈ V
    betweenness = nx.algorithms.centrality.betweenness_centrality(G)
    
    # Convert betweenness dict to list
    between_list = list(betweenness.values())
    
    # Convert networkx.Graph to Pyg Data
    pyg_data = from_networkx(G, between_list)
    print("\nEpoch: {}".format(epoch+1))
    
    

    
    ### Start training
    optimizer.zero_grad()

    pyg_data = pyg_data.to(setting.device)
    bc_pr = model(pyg_data)

    bc_gt = pyg_data.y
#     bc_gt = np.array(bc_gt)
#     bc_gt = torch.from_numpy(bc_gt)
    bc_gt = torch.reshape(bc_gt, (-1, ))

    # random sample 5|V| nodes
    src = (torch.rand(nodes_cnt * 5) * (nodes_cnt-1)).long()
    det = (torch.rand(nodes_cnt * 5) * (nodes_cnt-1)).long()

    y_gt = (bc_gt[det] - bc_gt[src]).squeeze().to(setting.device)
    y_pr = (bc_pr[det] - bc_pr[src]).squeeze()

    y_gt = nn.Sigmoid()(y_gt)
    y_pr = nn.Sigmoid()(y_pr)

    loss = nn.BCELoss(reduction="sum")(y_pr, y_gt)

    loss.backward()
    optimizer.step()
    
    
    ### Evaluation:- Train
    top1 = top_n(model, pyg_data, k=1)
    top5 = top_n(model, pyg_data, k=5)
    top10 = top_n(model, pyg_data, k=10)
    kendal = tau_distance(model, pyg_data)

    print("Train: loss = {}, top 1 = {}, top 5 = {}, top 10 = {}, kendal = {}".format(
            loss.item(),
            top1,
            top5,
            top10,
            kendal
    ))

    train_info["bce"].append(loss.item())
    train_info["top1"].append(top1)
    train_info["top5"].append(top5)
    train_info["top10"].append(top10)
    train_info["kendal"].append(kendal)

    with open(setting.train_info_p, 'w') as f:
        json.dump(train_info, f)



    ### Evaluation:- Synthetic
    bce_l = []
    top1_l = []
    top5_l = []
    top10_l = []
    kendal_l = []

    for data in all_loader:
        bce_l.append(eval_model(model, data))
        top1_l.append(top_n(model, data, k=1))
        top5_l.append(top_n(model, data, k=5))
        top10_l.append(top_n(model, data, k=10))
        kendal_l.append(tau_distance(model, data))
        
    bce = sum(bce_l)/len(bce_l)
    top1 = sum(top1_l)/len(top1_l)
    top5 = sum(top5_l)/len(top5_l)
    top10 = sum(top10_l)/len(top10_l)
    kendal = sum(kendal_l)/len(kendal_l)
        
    print("Valid: loss = {}, top 1 = {}, top 5 = {}, top 10 = {}, kendal = {}".format(
        bce,
        top1,
        top5,
        top10,
        kendal
    ))

    valid_info["bce"].append(bce)
    valid_info["top1"].append(top1)
    valid_info["top5"].append(top5)
    valid_info["top10"].append(top10)
    valid_info["kendal"].append(kendal)

    with open(setting.val_info_p, 'w') as f:
        json.dump(valid_info, f)
        
    
    if top10 > top_10:
        checkpoint = {
            'model_stat': model.state_dict(),
            'optimizer_stat': optimizer.state_dict(),
        }
        torch.save(checkpoint, setting.weight_drbc)
        top_10 = top10


Epoch: 1
Train: loss = 10397.207615964773, top 1 = 0.8666666666666667, top 5 = 0.38, top 10 = 0.2733333333333333, kendal = -0.07306213182171835
Valid: loss = 17328.6852710022, top 1 = 0.04400000000000002, top 5 = 0.03066666666666668, top 10 = 0.030333333333333344, kendal = -0.36601094618923774

Epoch: 2
Train: loss = 10397.201279889807, top 1 = 0.9333333333333333, top 5 = 0.8266666666666667, top 10 = 0.7866666666666666, kendal = 0.2689683227742581
Valid: loss = 17328.67719541735, top 1 = 0.45733333333333337, top 5 = 0.1594666666666667, top 10 = 0.10700000000000001, kendal = -0.26018311929052484

Epoch: 3
Train: loss = 10397.198070454247, top 1 = 0.9, top 5 = 0.8733333333333333, top 10 = 0.8033333333333333, kendal = 0.3877910414582639
Valid: loss = 17328.671764797484, top 1 = 0.6606666666666666, top 5 = 0.42973333333333336, top 10 = 0.32506666666666656, kendal = -0.08769764619590584

Epoch: 4
Train: loss = 10397.195731161795, top 1 = 0.9333333333333333, top 5 = 0.8266666666666667, top 


Epoch: 30
Train: loss = 10397.17712883892, top 1 = 0.9333333333333333, top 5 = 0.8666666666666667, top 10 = 0.8666666666666667, kendal = 0.20972724241413807
Valid: loss = 17328.661240362868, top 1 = 0.6546666666666667, top 5 = 0.6193333333333335, top 10 = 0.5222666666666668, kendal = -0.10217743281989734

Epoch: 31
Train: loss = 10397.176959700508, top 1 = 0.9666666666666667, top 5 = 0.86, top 10 = 0.83, kendal = 0.24894653773480052
Valid: loss = 17328.661182981105, top 1 = 0.6546666666666667, top 5 = 0.6194666666666667, top 10 = 0.5219333333333334, kendal = -0.10416226712009068

Epoch: 32
Train: loss = 10397.169175573552, top 1 = 0.9666666666666667, top 5 = 0.82, top 10 = 0.8266666666666667, kendal = 0.2191228187173502
Valid: loss = 17328.66030578234, top 1 = 0.6546666666666667, top 5 = 0.6193333333333335, top 10 = 0.523, kendal = -0.10523324931652996

Epoch: 33
Train: loss = 10397.175480941598, top 1 = 0.8666666666666667, top 5 = 0.8533333333333334, top 10 = 0.8066666666666666, kend


Epoch: 59
Train: loss = 10397.161403663438, top 1 = 0.9333333333333333, top 5 = 0.9133333333333333, top 10 = 0.79, kendal = 0.2804228076025342
Valid: loss = 17328.65585308939, top 1 = 0.654, top 5 = 0.6174666666666667, top 10 = 0.4642666666666667, kendal = -0.18178532773221315

Epoch: 60
Train: loss = 10397.16693620491, top 1 = 0.9666666666666667, top 5 = 0.8466666666666667, top 10 = 0.8266666666666667, kendal = 0.28738512837612534
Valid: loss = 17328.655627690365, top 1 = 0.654, top 5 = 0.6174666666666667, top 10 = 0.461, kendal = -0.1854181983063279

Epoch: 61
Train: loss = 10397.169281157672, top 1 = 0.8666666666666667, top 5 = 0.86, top 10 = 0.8433333333333334, kendal = 0.29439146382127374
Valid: loss = 17328.656120351214, top 1 = 0.654, top 5 = 0.6173333333333334, top 10 = 0.45786666666666664, kendal = -0.18919867173434693

Epoch: 62
Train: loss = 10397.168799392826, top 1 = 0.8666666666666667, top 5 = 0.86, top 10 = 0.81, kendal = 0.28597176836723354
Valid: loss = 17328.65607474

Valid: loss = 17328.652536789927, top 1 = 0.6539999999999999, top 5 = 0.618, top 10 = 0.4205333333333334, kendal = -0.2403612669200507

Epoch: 88
Train: loss = 10397.158748733238, top 1 = 0.9, top 5 = 0.9066666666666666, top 10 = 0.8333333333333334, kendal = 0.5180291208180504
Valid: loss = 17328.65542628153, top 1 = 0.6539999999999999, top 5 = 0.6166666666666668, top 10 = 0.41753333333333337, kendal = -0.24343683670067348

Epoch: 89
Train: loss = 10397.160563061623, top 1 = 0.8666666666666667, top 5 = 0.8466666666666667, top 10 = 0.8133333333333334, kendal = 0.487839057463599
Valid: loss = 17328.65593521772, top 1 = 0.6539999999999999, top 5 = 0.6161333333333334, top 10 = 0.4166, kendal = -0.24599021671000865

Epoch: 90
Train: loss = 10397.149214728652, top 1 = 0.8666666666666667, top 5 = 0.8333333333333334, top 10 = 0.7866666666666666, kendal = 0.45686562187395796
Valid: loss = 17328.65637476078, top 1 = 0.6539999999999999, top 5 = 0.6170666666666665, top 10 = 0.4167333333333334, ken


Epoch: 116
Train: loss = 10397.15101365724, top 1 = 0.9333333333333333, top 5 = 0.8533333333333334, top 10 = 0.8233333333333334, kendal = 0.6476505501833945
Valid: loss = 17328.653316972846, top 1 = 0.6539999999999999, top 5 = 0.6182666666666667, top 10 = 0.42366666666666664, kendal = -0.21907234246849372

Epoch: 117
Train: loss = 10397.144637531543, top 1 = 0.8666666666666667, top 5 = 0.8466666666666667, top 10 = 0.84, kendal = 0.6393655662998777
Valid: loss = 17328.652507706087, top 1 = 0.6539999999999999, top 5 = 0.6196000000000002, top 10 = 0.42726666666666663, kendal = -0.2165914516236581

Epoch: 118
Train: loss = 10397.156446680172, top 1 = 0.9333333333333333, top 5 = 0.8666666666666667, top 10 = 0.83, kendal = 0.6661976214293653
Valid: loss = 17328.654128650454, top 1 = 0.6539999999999999, top 5 = 0.6192000000000001, top 10 = 0.4273333333333333, kendal = -0.2169924651596986

Epoch: 119
Train: loss = 10397.15539313287, top 1 = 0.9, top 5 = 0.8333333333333334, top 10 = 0.83333333

...

Valid: loss = 17328.668016554922, top 1 = 0.6833333333333335, top 5 = 0.6675999999999999, top 10 = 0.6435333333333333, kendal = -0.008143735413749416

Epoch: 2619
Train: loss = 10397.122789913146, top 1 = 0.9333333333333333, top 5 = 0.94, top 10 = 0.91, kendal = 0.754114038012671
Valid: loss = 17328.66868307201, top 1 = 0.6833333333333335, top 5 = 0.6701333333333335, top 10 = 0.6536000000000002, kendal = 0.04037266119890645

Epoch: 2620
Train: loss = 10397.138858345443, top 1 = 1.0, top 5 = 0.9533333333333334, top 10 = 0.97, kendal = 0.7320831388240524
Valid: loss = 17328.67207431307, top 1 = 0.6833333333333335, top 5 = 0.6708000000000002, top 10 = 0.6546666666666667, kendal = 0.07722931786357272

Epoch: 2621
Train: loss = 10397.15423731555, top 1 = 0.9666666666666667, top 5 = 0.9533333333333334, top 10 = 0.9333333333333333, kendal = 0.7183394464821607
Valid: loss = 17328.668867723616, top 1 = 0.684, top 5 = 0.6717333333333334, top 10 = 0.6544666666666666, kendal = 0.09113258918450358


Valid: loss = 17328.67505121458, top 1 = 0.6833333333333335, top 5 = 0.6696000000000001, top 10 = 0.6537333333333335, kendal = 0.08514909915316399

Epoch: 2648
Train: loss = 10397.150957825444, top 1 = 0.9, top 5 = 0.9533333333333334, top 10 = 0.9433333333333334, kendal = 0.7616014226964544
Valid: loss = 17328.675002492477, top 1 = 0.6833333333333335, top 5 = 0.6699999999999999, top 10 = 0.6539999999999999, kendal = 0.06593929985997202

Epoch: 2649
Train: loss = 10397.130450005794, top 1 = 0.9333333333333333, top 5 = 0.9466666666666667, top 10 = 0.93, kendal = 0.753890852506391
Valid: loss = 17328.671735650674, top 1 = 0.684, top 5 = 0.6698666666666668, top 10 = 0.6504666666666667, kendal = 0.02266216443288658

Epoch: 2650
Train: loss = 10397.136827438228, top 1 = 0.9666666666666667, top 5 = 0.9333333333333333, top 10 = 0.9433333333333334, kendal = 0.6985559630988106
Valid: loss = 17328.67017221546, top 1 = 0.6846666666666668, top 5 = 0.6702666666666668, top 10 = 0.6384000000000002, ke

Valid: loss = 17328.67016496533, top 1 = 0.6846666666666668, top 5 = 0.6683999999999999, top 10 = 0.6235333333333333, kendal = -0.031833214642928584

Epoch: 2677
Train: loss = 10397.146168456413, top 1 = 1.0, top 5 = 0.9466666666666667, top 10 = 0.9533333333333334, kendal = 0.766537290207847
Valid: loss = 17328.67183536317, top 1 = 0.684, top 5 = 0.6690666666666668, top 10 = 0.6491333333333335, kendal = 0.01694898446355938

Epoch: 2678
Train: loss = 10397.153281444833, top 1 = 0.9666666666666667, top 5 = 0.94, top 10 = 0.9433333333333334, kendal = 0.7753242191841725
Valid: loss = 17328.676153647117, top 1 = 0.6833333333333335, top 5 = 0.6689333333333335, top 10 = 0.6526666666666667, kendal = 0.06276148563045943

Epoch: 2679
Train: loss = 10397.150302822323, top 1 = 1.0, top 5 = 0.9666666666666667, top 10 = 0.9333333333333333, kendal = 0.7905061687229076
Valid: loss = 17328.67353995784, top 1 = 0.684, top 5 = 0.6686666666666669, top 10 = 0.6517333333333333, kendal = 0.05877257051410282


Valid: loss = 17328.67076727187, top 1 = 0.684, top 5 = 0.6618666666666666, top 10 = 0.6430666666666667, kendal = 0.006267242781889712

Epoch: 2706
Train: loss = 10397.150754537943, top 1 = 1.0, top 5 = 0.9666666666666667, top 10 = 0.96, kendal = 0.7612635322885406
Valid: loss = 17328.67233364921, top 1 = 0.6833333333333335, top 5 = 0.6636000000000001, top 10 = 0.6480666666666667, kendal = 0.0345593598719744

Epoch: 2707
Train: loss = 10397.138134130984, top 1 = 0.9666666666666667, top 5 = 0.9533333333333334, top 10 = 0.95, kendal = 0.7642885406246527
Valid: loss = 17328.674614894666, top 1 = 0.6833333333333335, top 5 = 0.6646666666666668, top 10 = 0.6507333333333332, kendal = 0.04209498433019938

Epoch: 2708
Train: loss = 10397.126837704114, top 1 = 1.0, top 5 = 0.9466666666666667, top 10 = 0.9433333333333334, kendal = 0.7509085250639103
Valid: loss = 17328.674759140307, top 1 = 0.6826666666666668, top 5 = 0.6660000000000001, top 10 = 0.6489333333333335, kendal = 0.018746672001066878



Epoch: 2734
Train: loss = 10397.150819066837, top 1 = 1.0, top 5 = 0.96, top 10 = 0.9533333333333334, kendal = 0.7126953428920751
Valid: loss = 17328.669613615115, top 1 = 0.6826666666666668, top 5 = 0.6665333333333333, top 10 = 0.6452666666666664, kendal = 0.004496328599053145

Epoch: 2735
Train: loss = 10397.133969556231, top 1 = 0.9333333333333333, top 5 = 0.96, top 10 = 0.9366666666666666, kendal = 0.6022136267644771
Valid: loss = 17328.66649626713, top 1 = 0.6833333333333335, top 5 = 0.6669333333333333, top 10 = 0.6256666666666667, kendal = -0.028665674468226984

Epoch: 2736
Train: loss = 10397.155716683923, top 1 = 0.9333333333333333, top 5 = 0.96, top 10 = 0.92, kendal = 0.586920084472602
Valid: loss = 17328.66530684544, top 1 = 0.6826666666666668, top 5 = 0.6681333333333332, top 10 = 0.6156666666666666, kendal = -0.03835687137427487

Epoch: 2737
Train: loss = 10397.157138493643, top 1 = 0.9666666666666667, top 5 = 0.9466666666666667, top 10 = 0.95, kendal = 0.6723801267089029


# Visualization

In [ ]:
import matplotlib.pyplot as plt

with open(setting.train_info_p, 'r') as train_info, \
    open(setting.val_info_p, 'r') as valid_info:
    
    train = json.load(train_info)
    valid = json.load(valid_info)

    epochs = setting.epochs
    keys = list(train.keys())

    x = np.linspace(1, epochs, epochs)
    
    ### plot bce loss and mean top N %
    fig, axs = plt.subplots(len(keys), figsize=(25, 4 * len(keys)))
    for index, ax in enumerate(axs):
        key = keys[index]
        
        if key == "bce":
            y_train = [v/nodes_cnt for v in train[key]]
            y_valid = [v/5000 for v in valid[key]]
            ax.plot(x, y_train, color="blue")
            ax.plot(x, y_valid, color="orange")            
        else:
            ax.plot(x, train[key], color="blue")
            ax.plot(x, valid[key], color="orange")
        
        ax.legend(["train", "valid"], loc='upper left')
        ax.set_title(key)
        ax.grid()
        
    plt.tight_layout()
    plt.savefig(setting.result_plt_p, facecolor="white")

# Baseline

In [ ]:
import networkit as nk

In [ ]:
G = nk.readGraph("/hw1_data/Synthetic/5000/0.txt", nk.Format.EdgeListTabZero)

## RK (DIAM)

### ApproxBetweenness

In [ ]:
ab = nk.centrality.ApproxBetweenness(G, epsilon=0.1)
ab.run()

In [ ]:
# The 10 most central nodes according to betweenness are then 
ab.ranking()[:10]

## ARBA

## KADABRA

In [ ]:
# Initialize algorithm 
kadabra = nk.centrality.KadabraBetweenness(G, 0.05, 0.8) 
kadabra.run() 

In [ ]:
#The 10 most central nodes according to betweenness are then 
kadabra.ranking()[:10]

## K-BC (KPATH)

In [ ]:
kbc = nk.centrality.KPathCentrality(G, alpha=0.2, k=0)
kbc.run()

In [ ]:
kbc.ranking()[:10]